# Chapter 4

## 4.1.2

In [1]:
rdd_trans = sc.textFile('dataIn/ch04_data_transactions.txt')

In [2]:
rdd_trans.count()

1000

In [3]:
rdd_trans.take(5)

[u'2015-03-30#6:55 AM#51#68#1#9506.21',
 u'2015-03-30#7:39 PM#99#86#5#4107.59',
 u'2015-03-30#11:57 AM#79#58#7#2987.22',
 u'2015-03-30#12:46 AM#51#50#6#7501.89',
 u'2015-03-30#11:39 AM#86#24#5#8370.2']

In [4]:
rdd_trans_data = rdd_trans.map(lambda line: line.split('#'))

In [5]:
rdd_trans_data.count()

1000

In [6]:
rdd_trans_data.take(5)

[[u'2015-03-30', u'6:55 AM', u'51', u'68', u'1', u'9506.21'],
 [u'2015-03-30', u'7:39 PM', u'99', u'86', u'5', u'4107.59'],
 [u'2015-03-30', u'11:57 AM', u'79', u'58', u'7', u'2987.22'],
 [u'2015-03-30', u'12:46 AM', u'51', u'50', u'6', u'7501.89'],
 [u'2015-03-30', u'11:39 AM', u'86', u'24', u'5', u'8370.2']]

In [11]:
# create paired rdd
rdd_trans_by_cust = rdd_trans_data.map(lambda trans: (int(trans[2]), trans))

In [12]:
rdd_trans_by_cust.count()

1000

In [13]:
rdd_trans_by_cust.take(5)

[(51, [u'2015-03-30', u'6:55 AM', u'51', u'68', u'1', u'9506.21']),
 (99, [u'2015-03-30', u'7:39 PM', u'99', u'86', u'5', u'4107.59']),
 (79, [u'2015-03-30', u'11:57 AM', u'79', u'58', u'7', u'2987.22']),
 (51, [u'2015-03-30', u'12:46 AM', u'51', u'50', u'6', u'7501.89']),
 (86, [u'2015-03-30', u'11:39 AM', u'86', u'24', u'5', u'8370.2'])]

In [17]:
# Unique transactions
rdd_trans_by_cust.keys().distinct().count()

100

In [18]:
rdd_trans_by_cust.keys().distinct().take(5)

[2, 4, 6, 8, 10]

## Give a complimentary bear doll (ID 4) to the customer who made the most transactions.

In [38]:
# 1st method
(trans_count, cid) = sorted(rdd_trans_by_cust \
                            .groupByKey() \
                            .mapValues(len) \
                            .map(lambda t: (t[1], t[0])) \
                            .collect()
                           )[-1]
(trans_count, cid)

(19, 53)

In [39]:
# 2nd method
(cid, trans_count) = sorted(rdd_trans_by_cust \
                            .groupByKey() \
                            .mapValues(len) \
                            .collect(), key=lambda t: t[1], reverse=True)[0]
(cid, trans_count)

(53, 19)

In [40]:
(cid, trans_count) = sorted(rdd_trans_by_cust \
                            .countByKey() \
                            .items(), key=lambda t: t[1], reverse=True)[0]
(cid, trans_count)

(53, 19)

### Transactions made by customerId : 53

In [41]:
rdd_trans_by_cust.lookup(53)

[[u'2015-03-30', u'6:18 AM', u'53', u'42', u'5', u'2197.85'],
 [u'2015-03-30', u'4:42 AM', u'53', u'44', u'6', u'9182.08'],
 [u'2015-03-30', u'2:51 AM', u'53', u'59', u'5', u'3154.43'],
 [u'2015-03-30', u'5:57 PM', u'53', u'31', u'5', u'6649.27'],
 [u'2015-03-30', u'6:11 AM', u'53', u'33', u'10', u'2353.72'],
 [u'2015-03-30', u'9:46 PM', u'53', u'93', u'1', u'2889.03'],
 [u'2015-03-30', u'4:15 PM', u'53', u'72', u'7', u'9157.55'],
 [u'2015-03-30', u'2:42 PM', u'53', u'94', u'1', u'921.65'],
 [u'2015-03-30', u'8:30 AM', u'53', u'38', u'5', u'4000.92'],
 [u'2015-03-30', u'6:06 AM', u'53', u'12', u'6', u'2174.02'],
 [u'2015-03-30', u'3:44 AM', u'53', u'47', u'1', u'7556.32'],
 [u'2015-03-30', u'10:25 AM', u'53', u'30', u'2', u'5107.0'],
 [u'2015-03-30', u'1:48 AM', u'53', u'58', u'4', u'718.93'],
 [u'2015-03-30', u'9:31 AM', u'53', u'18', u'4', u'8214.79'],
 [u'2015-03-30', u'9:04 AM', u'53', u'68', u'4', u'9246.59'],
 [u'2015-03-30', u'1:51 AM', u'53', u'40', u'1', u'4095.5'],
 [u'2015-0

In [87]:
# Complementary Bear doll
l_l_complTrans = [["2015-03-30", "11:59 PM", "53", "4", "1", "0.00"]]

## Give a 5% discount for two or more Barbie Shopping Mall Playsets bought

### filter() when called on a paired rdd receives a tuple (k, v)

In [43]:
rdd_trans_by_cust.filter(lambda t: int(t[1][3]) == 25 and int(t[1][4]) > 1).count()

9

In [44]:
def isMultipleBarbiShpMallPlay(trans):
    return int(trans[3]) == 25 and int(trans[4]) > 1

In [53]:
rdd_trans_by_cust.filter(lambda t: isMultipleBarbiShpMallPlay(t[1])).take(10)

[(17, [u'2015-03-30', u'6:26 PM', u'17', u'25', u'6', u'7193.11']),
 (93, [u'2015-03-30', u'7:27 AM', u'93', u'25', u'7', u'2749.15']),
 (68, [u'2015-03-30', u'1:07 AM', u'68', u'25', u'9', u'8391.61']),
 (59, [u'2015-03-30', u'1:23 AM', u'59', u'25', u'5', u'5296.69']),
 (42, [u'2015-03-30', u'9:45 AM', u'42', u'25', u'10', u'1363.97']),
 (77, [u'2015-03-30', u'10:40 PM', u'77', u'25', u'3', u'3345.81']),
 (22, [u'2015-03-30', u'12:53 PM', u'22', u'25', u'9', u'6996.42']),
 (32, [u'2015-03-30', u'12:44 AM', u'32', u'25', u'8', u'8849.5']),
 (75, [u'2015-03-30', u'5:30 PM', u'75', u'25', u'10', u'3557.01'])]

In [48]:
def apply5PercentDiscount(trans):
    if isMultipleBarbiShpMallPlay(trans):
        trans[5] = str(float(trans[5]) * 0.95)
    return trans

In [50]:
rdd_trans_by_cust_discount = rdd_trans_by_cust.mapValues(lambda trans: apply5PercentDiscount(trans))

In [52]:
rdd_trans_by_cust_discount.filter(lambda t: isMultipleBarbiShpMallPlay(t[1])).take(10)

[(17, [u'2015-03-30', u'6:26 PM', u'17', u'25', u'6', '6833.4545']),
 (93, [u'2015-03-30', u'7:27 AM', u'93', u'25', u'7', '2611.6925']),
 (68, [u'2015-03-30', u'1:07 AM', u'68', u'25', u'9', '7972.0295']),
 (59, [u'2015-03-30', u'1:23 AM', u'59', u'25', u'5', '5031.8555']),
 (42, [u'2015-03-30', u'9:45 AM', u'42', u'25', u'10', '1295.7715']),
 (77, [u'2015-03-30', u'10:40 PM', u'77', u'25', u'3', '3178.5195']),
 (22, [u'2015-03-30', u'12:53 PM', u'22', u'25', u'9', '6646.599']),
 (32, [u'2015-03-30', u'12:44 AM', u'32', u'25', u'8', '8407.025']),
 (75, [u'2015-03-30', u'5:30 PM', u'75', u'25', u'10', '3379.1595'])]

## Add a complimentary toothbrush (ID 70) to customers who bought five or more dictionaries (ID 81)

In [57]:
def isMultipleDictBought(tran):
    return int(tran[3]) == 81 and int(tran[4]) > 4

In [56]:
rdd_trans_by_cust.filter(lambda t: isMultipleDictBought(t[1])).take(10)

[(85, [u'2015-03-30', u'1:55 PM', u'85', u'81', u'7', u'9648.24']),
 (77, [u'2015-03-30', u'9:34 PM', u'77', u'81', u'5', u'7798.28']),
 (82, [u'2015-03-30', u'3:40 AM', u'82', u'81', u'6', u'3665.45']),
 (47, [u'2015-03-30', u'10:39 PM', u'47', u'81', u'10', u'1122.89']),
 (16, [u'2015-03-30', u'11:02 AM', u'16', u'81', u'9', u'9063.74']),
 (10, [u'2015-03-30', u'2:54 PM', u'10', u'81', u'10', u'9897.61'])]

In [61]:
def addToothBrush(tran):
    if isMultipleDictBought(tran):
        from copy import copy
        cloned_tran = copy(tran)
        cloned_tran[5] = "0.00"
        cloned_tran[3] = "70"
        cloned_tran[4] = "1"
        return [tran, cloned_tran]
    else:
        return [tran]

In [76]:
rdd_trans_by_cust_with_comp = rdd_trans_by_cust.flatMapValues(lambda tran: addToothBrush(tran))

In [63]:
rdd_trans_by_cust_with_comp.count()

1006

In [64]:
rdd_trans_by_cust_with_comp.filter(lambda t: isMultipleDictBought(t[1])).take(10)

[(85, [u'2015-03-30', u'1:55 PM', u'85', u'81', u'7', u'9648.24']),
 (77, [u'2015-03-30', u'9:34 PM', u'77', u'81', u'5', u'7798.28']),
 (82, [u'2015-03-30', u'3:40 AM', u'82', u'81', u'6', u'3665.45']),
 (47, [u'2015-03-30', u'10:39 PM', u'47', u'81', u'10', u'1122.89']),
 (16, [u'2015-03-30', u'11:02 AM', u'16', u'81', u'9', u'9063.74']),
 (10, [u'2015-03-30', u'2:54 PM', u'10', u'81', u'10', u'9897.61'])]

In [70]:
def isCompToothBrush(tran):
    return int(tran[3]) == 70 and float(tran[5]) == 0.0

In [73]:
rdd_trans_by_cust_with_comp.filter(lambda t: isCompToothBrush(t[1])).take(10)

[(85, [u'2015-03-30', u'1:55 PM', u'85', '70', '1', '0.00']),
 (77, [u'2015-03-30', u'9:34 PM', u'77', '70', '1', '0.00']),
 (82, [u'2015-03-30', u'3:40 AM', u'82', '70', '1', '0.00']),
 (47, [u'2015-03-30', u'10:39 PM', u'47', '70', '1', '0.00']),
 (16, [u'2015-03-30', u'11:02 AM', u'16', '70', '1', '0.00']),
 (10, [u'2015-03-30', u'2:54 PM', u'10', '70', '1', '0.00'])]

## Give a pair of pajamas (ID 63) to the customer who spent the most over all

In [77]:
# First convert transaction amount to float
rdd_amount_by_cust = rdd_trans_by_cust_with_comp.mapValues(lambda tran: float(tran[5]))

In [78]:
rdd_amount_by_cust.take(5)

[(51, 9506.21), (99, 4107.59), (79, 2987.22), (51, 7501.89), (86, 8370.2)]

In [79]:
l_total = rdd_amount_by_cust.foldByKey(0, lambda v1, v2: v1 + v2).collect()

In [81]:
len(l_total)

100

In [82]:
(cid, total) = sorted(l_total, key=lambda t: t[1], reverse=True)[0]

In [83]:
(cid, total)

(76, 100049.0)

In [88]:
# Complementary pajamas
l_l_complTrans += [["2015-03-30", "11:59 PM", "76", "63", "1", "0.00"]]

### Add complementary transactions to original rdd and save it to a file
### Comp trans could have been added using foldByKey() as done above but this is another method

In [89]:
l_l_complTrans

[['2015-03-30', '11:59 PM', '53', '4', '1', '0.00'],
 ['2015-03-30', '11:59 PM', '76', '63', '1', '0.00']]

In [90]:
# Create rdd from comp trans
rdd_comp_trans = sc.parallelize(l_l_complTrans) \
                    .map(lambda tran: (int(tran[2]), tran))

In [91]:
rdd_comp_trans.take(5)

[(53, ['2015-03-30', '11:59 PM', '53', '4', '1', '0.00']),
 (76, ['2015-03-30', '11:59 PM', '76', '63', '1', '0.00'])]

In [92]:
rdd_final_trans_by_cust = rdd_trans_by_cust_with_comp.union(rdd_comp_trans)

In [93]:
rdd_final_trans_by_cust.count()

1008

In [95]:
# Save the rdd
rdd_final_trans_by_cust.map(lambda t: '#'.join(t[1])).saveAsTextFile('dataOut/ch04_finaltransbycust')

### [aggregateByKey](http://spark.apache.org/docs/2.1.0/api/python/pyspark.html#pyspark.RDD.aggregateByKey)